# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet') # download for lemmatization
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, classification_report
from workspace_utils import active_session
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database where the cleaned and transformed data was saved
engine = create_engine('sqlite:///DisasterResponse.db')
df =  pd.read_sql_table('df_Message', con=engine)
X = df['message']
Y = df.drop(columns=['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
# This is t stem the raww text
def tokenize(text):
    """Normalize, tokenize and stem text string
    
    Args:
    text: string. String containing message for processing
       
    Returns:
    stemmed: list of strings. List containing normalized and stemmed word tokens
    """
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    words = text.split()
    words = [w for w in words if w not in stopwords.words('english')]
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# Building of the pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# splitting the data into test and tran sets and fitting the pipeline
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
# predict the model
labels = Y.columns.values
Y_pred = pipeline.predict(X_test)
# Y_pred = pd.DataFrame(Y_pred, columns = labels)

In [7]:
Y_pred.shape

(5206, 35)

In [8]:
X_test.shape

(5206,)

In [9]:
# for i in range(len(labels)):
#     print(classification_report(Y_test.iloc[:,i], Y_pred.iloc[:,i], target_names=labels[i]))
#     print(i)

In [11]:
# this is a module to show the required data, this was used for the original as well as the improved pipelines
def show_results(test, pred, labels):
    """Show the results for the accuracy, precision, recall and F1 score.
    
    Args:
    test: df. This is the dataframe that contains the test data.
    pred: series. This is a series with the predicted values.
    lables: series. a List of the column names which is the categories in the categories dataset.
       
    Returns:
    df_results: dataframe. Dataframe containing results dataset.
    """
    pred = pd.DataFrame(pred, columns = labels)
    results = []
    for i in range(len(labels)):
        f1 = f1_score(test.iloc[:,i], pred.iloc[:,i], average='weighted')
        precision = precision_score(test.iloc[:,i], pred.iloc[:,i], average='weighted')
        recall = recall_score(test.iloc[:,i], pred.iloc[:,i], average='weighted')
        accuracy = accuracy_score(test.iloc[:,i], pred.iloc[:,i])
        results.append([f1, precision, recall, accuracy])

    array_results = np.array(results)
    df_results = pd.DataFrame(data=array_results, index = labels, columns = ['Precision', 'Recall', 'F1', 'Accuracy']) 
    return df_results

In [12]:
# shoe the results
show_results(Y_test, Y_pred, labels)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Precision,Recall,F1,Accuracy
related,0.817711,0.814925,0.826738,0.826738
request,0.868709,0.877828,0.883788,0.883788
offer,0.994529,0.992714,0.996350,0.996350
aid_related,0.740564,0.748815,0.746638,0.746638
medical_help,0.892627,0.903821,0.919900,0.919900
medical_products,0.933694,0.939603,0.950634,0.950634
search_and_rescue,0.960626,0.965612,0.971955,0.971955
security,0.967940,0.957803,0.978294,0.978294
military,0.957843,0.958155,0.968498,0.968498
water,0.947849,0.951613,0.955820,0.955820


In [13]:
pipeline.get_params()

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
with active_session(): 
    parameters = {
                  'tfidf__use_idf': [True, False],
                  'clf__estimator__n_estimators': [10, 14],
                  'vect__max_df': [1, 6]
                  }

    cv =  GridSearchCV(pipeline, param_grid = parameters)
    improve_model = cv.fit(X_train, Y_train)
    Y_pred_imp = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
show_results(Y_test, Y_pred_imp, labels)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Precision,Recall,F1,Accuracy
related,0.690733,0.676458,0.768536,0.768536
request,0.758371,0.759116,0.821744,0.821744
offer,0.994529,0.992714,0.996350,0.996350
aid_related,0.486170,0.566623,0.579524,0.579524
medical_help,0.873218,0.851243,0.909143,0.909143
medical_products,0.920368,0.904709,0.941990,0.941990
search_and_rescue,0.955772,0.946295,0.968498,0.968498
security,0.967364,0.957779,0.977142,0.977142
military,0.952103,0.944306,0.965232,0.965232
water,0.906690,0.892638,0.931809,0.931809


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
with active_session():  
    pipeline_new = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

    parameters_new = {'tfidf__use_idf': [True, False],
                      'clf__estimator__n_estimators': [10, 14],
                      'vect__max_df': [1, 6]
                      }

    cv_new =  GridSearchCV(pipeline_new, param_grid = parameters_new)

    improve_model_new = cv_new.fit(X_train, Y_train)
    Y_pred_new = cv_new.predict(X_test)

In [17]:
show_results(Y_test, Y_pred_new, labels)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Precision,Recall,F1,Accuracy
related,0.682103,0.606699,0.778909,0.778909
request,0.745828,0.855512,0.824818,0.824818
offer,0.994337,0.992713,0.995966,0.995966
aid_related,0.416303,0.327221,0.572032,0.572032
medical_help,0.873411,0.835999,0.914330,0.914330
medical_products,0.921102,0.896770,0.946792,0.946792
search_and_rescue,0.955943,0.942074,0.970227,0.970227
security,0.967940,0.957803,0.978294,0.978294
military,0.951382,0.936117,0.967153,0.967153
water,0.905385,0.876847,0.935843,0.935843


In [18]:
improve_model.best_params_

{'clf__estimator__n_estimators': 14, 'tfidf__use_idf': True, 'vect__max_df': 1}

In [19]:
improve_model_new.best_params_

{'clf__estimator__n_estimators': 10, 'tfidf__use_idf': True, 'vect__max_df': 1}

### 9. Export your model as a pickle file

In [20]:
# Pickle best model
pickle.dump(improve_model_new, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# used this cell to modular the code for the IDE environment 
import sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV

def load_data(database_filepath):
    # Load data from database
    engine = create_engine('sqlite:///' + database_filepath)
    df =  pd.read_sql_table('df_Message', con=engine)
    
    # Create X and Y datasets
    X = df['message']
    Y = df.drop(columns=['id', 'message', 'original', 'genre'], axis=1)
    
    # Create list containing all category names
    category_names = Y.columns.values
    
    return X, Y, category_names


def build_model():
    # Create pipeline
    pipeline = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultiOutputClassifier(AdaBoostClassifier()))
                        ])    
    # Create parameters dictionary
    parameters =  {
                  'tfidf__use_idf': [True, False],
                  'clf__estimator__n_estimators': [10, 14],
                  'vect__max_df': [1, 6]
                  }    
    Create grid search object
    cv = GridSearchCV(pipeline, param_grid = parameters)
#     cv = pipeline
    return cv

def evaluate_model(model, X_test, Y_test):
    # Predict labels for test dataset
    Y_pred = model.predict(X_test)
    return Y_pred


def save_model(model, model_filepath):
    #save the model     
    pickle.dump(model, open(model_filepath, 'wb'))

In [ ]:
X, Y, category_names = load_data('DisasterResponse.db')
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
Y_train.shape

In [ ]:
Y_test.shape

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
pipeline.fit(X_train, Y_train)
# labels = Y.columns.values
Y_pred = pipeline.predict(X_test)

In [ ]:
model = build_model()
model.fit(X_train, Y_train)

In [ ]:
evaluate_model(model, X_test, Y_test)

In [ ]:
Y_pred.shape

In [ ]:
X_test.shape

In [ ]:
save_model(model, 'classifier.pkl')